## Redes Neurais Artificiais 2025.1

- **Disciplina**: Redes Neurais Artificiais 2025.1  
- **Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  
- **Github**: http://github.com/elloa  
        

Levando em conta a base de dados **_Forest Cover Type_**, esta segunda parte do Projeto Prático 2.2 diz respeito à proposição e avaliação de múltiplas redes neurais artificiais do tipo feedforward multilayer perceptron para o problema da classificação multi-classe da cobertura florestal em uma área do Roosevelt National Forest.

## Testando Redes Neurais sem os Atributos Categórios

1. Abra a base de dados em questão
2. Elimine todas as colunas relativas aos atributos categóricos
3. Armazene o atributo alvo em uma variável y e os atributos preditores em uma variável X
4. Efetue uma partição holdout 70/30 com o sklearn, distribuindo os exemplos de maneira aleatória
5. Efetue o escalonamento dos atributos

### Escalonando os atributos

O treinamento de uma rede neural artificial é mais eficiente quando os valores que lhes são fornecidos como entrada são pequenos, pois isto favorece a convergência. Isto é feito escalonando-se todos os atributos para o intervalo [0,1], mas precisa ser feito de maneira cautelosa, para que informações do conjunto de teste não sejam fornecidas no treinamento.

Há duas estratégias para tal escalonamento: normalização e padronização. Ambas possuem características particulares, vantagens e limitações, como é possível ver aqui: https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/


No nosso caso, vamos usar a padronização. Assim, com os atributos preditores do treinamento, isto é, X_train, deve-se subtrair a média e dividir pelo desvio padrão:

X_train_std = (X_train - np.mean(X_train))/np.std(X_train)

Em seguida, o mesmo deve ser feito com os atributos preditores do conjunto de testes, mas com padronização relativa ao conjunto de treinamento:

X_test_std = (X_test - np.mean(X_train))/np.std(X_train)

Se todo o conjunto X for utilizado na padronização, a rede neural receberá informações do conjunto de teste por meio da média e variância utilizada para preparar os dados de treinamento, o que não é desejável.


### Continuando

5. Treine uma rede neural multilayer perceptron para este problema com uma única camada e dez neurônios  
    5.1 Utilize a função de ativação ReLU  
    5.2 Utilize o solver Adam    
    5.3 Imprima o passo a passo do treinamento    
    5.4 Utilize o número máximo de épocas igual a 300  
6. Com o modelo em questão, após o treinamento, apresente:  
    6.1 Matriz de confusão para o conjunto de teste  
    6.2 Acurácia  
    6.3 F-Score  
    6.4 Precisão  
    6.5 Revocação  
7. Repita o treinamento da mesma rede anterior sem imprimir o passo a passo (verbose False) por 100 vezes  
    7.1 Cada uma destas repetições deve ser feita com uma nova partição Holdout  
    7.2 Apresente a média e o desvio padrão da acurácia e do F-Score para o conjunto de treino  
8. Repita por 100 vezes o treinamento desta mesma rede, mas utilizando o otimizador SGD  
    8.1 Apresente a média e o desvio padrão da acurácia e do F-Score para o conjunto de treino  
9. Houve influência da escolha do otimizador no desempenho da rede?

**Importando bibliotecas**

In [6]:
# Bibliotecas essenciais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Para o holdout
from sklearn.model_selection import train_test_split

# Métricas de avaliação
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
from prettytable import PrettyTable

# Classificador
from sklearn.neural_network import MLPClassifier

# Processamento paralelo para o treinamento
from joblib import Parallel, delayed

**Importando o dataset**

In [18]:
file_path = "covtype.csv"
pd.set_option('display.max_columns', None)
             
df_cat = pd.read_csv(file_path) #Dataset com atributos categóricos
df_cat.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


**Retirando os atributos categóricos**

In [19]:
categoric_cols = [col for col in df_cat.columns if 'Soil_Type' in col or 'Wilderness_Area' in col]
df = df_cat.drop(columns=categoric_cols)
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,5
1,2590,56,2,212,-6,390,220,235,151,6225,5
2,2804,139,9,268,65,3180,234,238,135,6121,2
3,2785,155,18,242,118,3090,238,238,122,6211,2
4,2595,45,2,153,-1,391,220,234,150,6172,5


**Definindo variável y e atributos preditores X:**

In [9]:
y = df.pop(item='Cover_Type')
X = df

**1. Holdout 70/30 normalizado**

**2. Primeiro treinamento**

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_std = (X_train - np.mean(X_train, axis=0))/np.std(X_train)
X_test_std = (X_test - np.mean(X_train, axis=0))/np.std(X_train)

#  Função de ativação ReLU e solver Adam implícitos
clf = MLPClassifier(hidden_layer_sizes=(10,), verbose=True, random_state=42, max_iter=300)
clf.fit(X_train_std, y_train)

C:\Users\Samsung\anaconda3\envs\rna_pp2\lib\site-packages\numpy\_core\fromnumeric.py:3800: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


Iteration 1, loss = 0.96560008
Iteration 2, loss = 0.70503840
Iteration 3, loss = 0.68408310
Iteration 4, loss = 0.67646322
Iteration 5, loss = 0.67186530
Iteration 6, loss = 0.66877967
Iteration 7, loss = 0.66670678
Iteration 8, loss = 0.66516924
Iteration 9, loss = 0.66414109
Iteration 10, loss = 0.66338850
Iteration 11, loss = 0.66275153
Iteration 12, loss = 0.66210352
Iteration 13, loss = 0.66151865
Iteration 14, loss = 0.66105187
Iteration 15, loss = 0.66054774
Iteration 16, loss = 0.65994690
Iteration 17, loss = 0.65924597
Iteration 18, loss = 0.65842715
Iteration 19, loss = 0.65775438
Iteration 20, loss = 0.65725450
Iteration 21, loss = 0.65671759
Iteration 22, loss = 0.65638514
Iteration 23, loss = 0.65610437
Iteration 24, loss = 0.65560265
Iteration 25, loss = 0.65535184
Iteration 26, loss = 0.65505213
Iteration 27, loss = 0.65483372
Iteration 28, loss = 0.65458697
Iteration 29, loss = 0.65454296
Iteration 30, loss = 0.65435427
Iteration 31, loss = 0.65422392
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(10,), max_iter=300, random_state=42,
              verbose=True)

**Métricas de avaliação o conjunto de Teste**
- Matriz de confusão
- Métricas: Acurácia, Fscore, precisão e revocação

In [ ]:
y_test_pred = clf.predict(X_test_std)

cm_test = confusion_matrix(y_test, y_test_pred, labels=[1, 2, 3, 4, 5, 6, 7])

# Plotando
fig, ax = plt.subplots()

# Matriz de Teste
ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=[1, 2, 3, 4, 5, 6, 7]).plot(ax=ax, cmap="Blues")
ax.set_title("Matriz de Confusão - Teste")

plt.tight_layout()
plt.show()

test_metrics = {
    "accuracy" : accuracy_score(y_test, y_test_pred),
    "precision" : precision_score(y_test, y_test_pred, average="weighted", zero_division=0),
    "recall" : recall_score(y_test, y_test_pred, average="weighted", zero_division=0),
    "f_1Score" : f1_score(y_test, y_test_pred, average="weighted", zero_division=0)
    }

table = PrettyTable()
table.padding_width = 8

table.title = "Métricas de teste"
table.field_names = ["Métricas", "Valor"]
table.add_row(["Acurácia", f"{test_metrics['accuracy']:.4f}"])
table.add_row(["Precisão", f"{test_metrics['precision']:.4f}"])
table.add_row(["Revocação", f"{test_metrics['recall']:.4f}"])
table.add_row(["F1_Score", f"{test_metrics['f_1Score']:.4f}"])

print("\n")
print(table)

**Rotina de avaliação de modelo com n treinamentos**

In [ ]:
def avaliar_modelo(X, y, camadas, solver, activation, iterations, n_times, resultados):
    f1_scores = []
    acuracias = []
    print(f"Arquitetura com solver={solver}, camadas={camadas} e n° de iterações={iterations}")
    
    for i in range(1, n_times + 1):
        print(f"Execução {i}/{n_times}")

        # Holdout
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i)

        # Normalização
        X_train_std = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
        X_test_std = (X_test - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)

        # Treinamento
        clf = MLPClassifier(hidden_layer_sizes=camadas, verbose=False, random_state=i,
                            max_iter=iterations, solver=solver, activation=activation)
        clf.fit(X_train_std, y_train)

        # Predição
        y_test_pred = clf.predict(X_test_std)

        # Guardando métricas
        f1_scores.append(f1_score(y_test, y_test_pred, average="weighted", zero_division=0))
        acuracias.append(accuracy_score(y_test, y_test_pred))

    media_acuracia = np.mean(acuracias)
    desvio_acuracia = np.std(acuracias)
    media_f1 = np.mean(f1_scores)
    desvio_f1 = np.std(f1_scores)

    print(f"Acurácia -> Média {media_acuracia:.4f} | Desvio padrão: {desvio_acuracia:.4f}")
    print(f"F1 score -> Média {media_f1:.4f} | Desvio padrão: {desvio_f1:.4f}")

    resultados.append({
        'camadas': camadas,
        'solver': solver,
        'activation': activation,
        'iterations': iterations,
        'media_acuracia': media_acuracia,
        'desvio_acuracia': desvio_acuracia,
        'media_f1': media_f1,
        'desvio_f1': desvio_f1
    })

1. **Repetindo o treinamento anterior 100 vezes com outras partições Hold-Out**
2. **Repetindo o treinamento anterior 100 vezes com outras partições Hold-Out, porém utilizando o solver='sgd'**

In [ ]:
resultados = []

# 1
print("\n--- Resultados Adam ---")
avaliar_modelo(X, y,camadas=(10,), solver='adam',activation='relu', iterations=300, n_times=100, resultados=resultados)

# 2
print("\n--- Resultados SGD ---")
avaliar_modelo(X, y,camadas=(10,), solver='sgd',activation='relu', iterations=300, n_times=100, resultados=resultados)

print(resultados)

## Discussão

Nos passos anteriores, você avaliou o desempenho de uma única rede neural que contém os seguintes parâmetros: uma única camada oculta com 10 neurônios e função de ativação ReLU. O otimizador utilizado, quer seja SGD ou ADAM, trata-se do algoritmo para aproximar o gradiente do erro. Neste sentido, a escolha do otimizador é um hiperparâmetro, pois diz respeito a como a rede neural definida previamente atuará "em tempo de execução"  durante o processo de treinamento. Também são hiperparâmetros a quantidade de épocas, a taxa de aprendizado inicial, dentre outros.

Cabe alientar também que você efetuou o treinamento desta rede por 100 vezes e apresentou os resultados em termos de média +- desvio padrão. Lembre-se que em uma rede neural há a inicialização aleatória de pesos e, em consequência, o desempenho delas está sujeito à uma flutuação estocástica. A execução destas múltiplas vezes faz com que eliminemos algum viés introduzido por uma boa ou má "sorte" na escolha de pesos no caso de uma única execução.

Você também aprendeu uma estratégia para escalonar os atributos para uma melhor convergência da rede. Utilize-a em todos os treinamentos e testes propostos a seguir.

## Propondo Novas Arquiteturas

Variando  os parâmetros (uma ou duas camadas ocultas, com diferente números de neurônios em cada uma delas e a função de ativação) e o hiperparâmetros solver (Adam ou SGD) e o número de épocas (100,150 e 200), atenda ao que se pede:

1. Proponha 10 arquiteturas distintas de RNAs para o problema em questão, à sua escolha
2. Avalie cada uma das arquiteturas perante todos os hiperparâmetros apresentados por 100 vezes
3. Como resultado da avaliação, apresente:  
    3.1 Top-3 melhores redes no tocante à F-Score e Acurácia  
    3.2 Repetição em que houve o melhor desempenho de cada uma dessas redes: ilustre tp, tf, fp e fn  

**Proposição de 10 arquiteturas de RNAs distintas**

Parâmetros e hiperparâmetros característicos:

* Quantidade de camadas
* Número de neurônios e por camada
* Solver
* Função de ativação
* Número de épocas

In [12]:
resultados = []

arquiteturas_propostas = [
    ((20,), 'adam', 'relu', 100),       #1
    ((20, 20), 'adam', 'relu', 150),    #2
    ((11, 2), 'adam', 'tanh', 100),     #3
    ((16, 32), 'adam', 'relu', 150),    #4
    ((5, 20), 'sgd', 'tanh', 150),      #5
    ((48, 60), 'sgd', 'relu', 150),     #6
    ((19, 40), 'sgd', 'logistic', 100), #7
    ((32, 32), 'sgd', 'relu', 150),     #8
    ((36, 28), 'adam', 'logistic', 150),#9
    ((32, 16), 'sgd', 'relu', 150),     #10
]

Dada a elevada demanda de processamento exigida para o treinamento repetitivo de tantas RNAs, fez-se uma execução em diferentes instâncias do presente jupyter notebook. As execuções em diferentes dispositivos utilizaram da biblioteca joblib para o processamento paralelo, tornando possível o treinamento simultâneo de redes diferentes e armazenando as métricas associadas em um dicionário inserido à lista "resultados".

In [ ]:
# O treinamento foi executado com o seguinte comando. Em cada instância, a lista "arquiteturas_propostas" continha alguns dos elementos listados na célula acima

# resultados = (Parallel(n_jobs=-1)(delayed(avaliar_modelo)(X, y, camadas, solver, activation, iterations, n_times=100) for (camadas, solver, activation, iterations) in arquiteturas_propostas))

# with open("resultado.json", "w", encoding="utf-8") as f:
#     json.dump(resultados, f, ensure_ascii=False, indent=4)

**TOP 3 ARQUITETURAS PROPOSTAS**

1 - (48,60), sgd, relu, 150

2 - (36,28), adam, logistic, 150

3 - (32,32), sgd, relu, 150

**Repetição de melhor desempenho**
<h1>ESSA ETAPA ESTÁ FALTANDO

## Estimando o número de neurônios

Um dos problemas de pesquisa com redes neurais artificiais consiste na determinação do número de neurônios em sua arquitetura. Embora não seja possível definir a priori qual rede neural é adequada para um problema, pois isto só é possível mediante uma busca exaustiva, há regras na literatura que sugerem o número de neurônios escondidos, tal como a regra da Pirâmide Geométrica, dada a seguir:

$$N_h = \alpha \cdot \sqrt{N_i \cdot N_o},$$

em que $N_h$ é o número de neurônios ocultos (a serem distribuídos em uma ou duas camadas ocultas), $N_i$ é o número de neurônios na camada de entrada e $N_o$ é o número de neurônios na camada de saída. 

1. Consulte a documentação da classe MLPClassifier (disponível em https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) e obtenha os valores de $N_i$ e $N_o$.
2. Teste os valores de $\alpha$ como sendo iguais a $0.5$, $2$ e $3$.
3. Proponha pelo menos 30 redes neurais segundo a regra da pirâmide geométrica e teste-as nos mesmos termos estabelecidos anterioremente  (solver, épocas, etc.)  
    3.1 Apresente as top-3 melhores redes no tocante à F-Score e Acurácia  

In [15]:
alphas = [0.5, 2, 3]
nhs = []

# Definindo ni = Número de neurônios na camada de entrada (Número de features em X)
ni = 10 

# Definindo no = Número de neurônios na camada de saída (Número de tipos em Cover_Type)
no = 7

for alpha in alphas:
    nh = (alpha * ((ni * no)** 0.5))
    nh = int(round(nh, 0))
    nhs.append(nh)

print("Número de neurônios para a camada oculta de acordo com a pirâmide geométrica:", nhs)

Número de neurônios para a camada oculta de acordo com a pirâmide geométrica: [4, 17, 25]


O processo de treinamento das arquiteturas propostas se deu do mesmo modo que foi estabelecido anteriormente (com múltiplos dispositivos e processamento paralelo. As arquiteturas definidas estão listadas a seguir.

In [16]:
# Lista com todas as arquiteturas propostas segundo a pirâmide geométrica
resultados = []
arquiteturas = [
    # 4 neurônios
    ((4,), 'adam', 'tanh', 100),        #1
    ((4,), 'sgd', 'relu', 150),         #2
    ((2, 2), 'adam', 'tanh', 150),      #3
    ((1, 3), 'adam', 'relu', 150),      #4
    ((3, 1), 'adam', 'logistic', 100),  #5
    ((2, 2), 'sgd', 'logistic', 150),   #6
    ((1, 3), 'sgd', 'logistic', 150),   #7
    ((3, 1), 'sgd', 'relu', 150),       #8

    # 17 neurônios
    ((12, 5), 'adam', 'tanh', 150),     #9
    ((5, 12), 'adam', 'logistic', 150), #10
    ((9, 8), 'adam', 'relu', 150),      #11
    ((8, 9), 'adam', 'logistic', 100),  #12
    ((11, 6), 'adam', 'relu', 150),     #13
    ((6, 11), 'adam', 'tanh', 150),     #14
    ((12, 5), 'sgd', 'relu', 100),      #15
    ((5, 12), 'sgd', 'relu', 150),      #16
    ((9, 8), 'sgd', 'tanh', 100),       #17
    ((8, 9), 'sgd', 'relu', 150),       #18
    ((11, 6), 'sgd', 'logistic', 150),  #19
    ((6, 11), 'sgd', 'relu', 100),      #20

    # 25 neurônios
    ((10, 15), 'adam', 'relu', 100),    #21
    ((15, 10), 'sgd', 'relu', 150),     #22
    ((12, 13), 'adam', 'relu', 100),    #23
    ((13, 12), 'sgd', 'relu', 150),     #24
    ((8, 17), 'sgd', 'tanh', 150),      #25
    ((17, 8), 'adam', 'logistic', 100), #26
    ((17, 8), 'sgd', 'relu', 150),      #27
    ((20, 5), 'adam', 'relu', 100),     #28
    ((5, 20), 'sgd', 'relu', 150),      #29
    ((8, 17), 'adam', 'relu', 150),     #30
]

In [ ]:
# O treinamento foi executado com o seguinte comando. Em cada instância, a lista "arquiteturas" continha alguns dos elementos listados na célula acima

# resultados = (Parallel(n_jobs=-1)(delayed(avaliar_modelo)(X, y, camadas, solver, activation, iterations, n_times=100) for (camadas, solver, activation, iterations) in arquiteturas))

# with open("resultado_arquiteturas.json", "w", encoding="utf-8") as f:
#     json.dump(resultados, f, ensure_ascii=False, indent=4)

**TOP 3 ARQUITETURAS PROPOSTAS**

1 - (17,8), adam, logistic, 100

2 - (12, 13), adam, relu, 100

3 - (15, 10), adam, relu, 100

## Testando as Redes Neurais com Atributos Categóricos

1. Considere as 6 redes neurais obtidas nos dois top-3 anteriores (arquiteturas próprias e regra da pirâmide geométrica)
2. Com todos os atributos preditores da base de dados original, incluindo os categóricos, treine e teste estas mesmas redes por 100 repetições  
    2.1 Considere o melhor otimizador para cada uma delas  
    2.2 Faça uso de 200 épocas para treinamento  
    2.2 Apresente os resultados de acurácia e F-Score em termos da média +- dp para cada arquitetura
3. Apresente o gráfico boxplot para o F-Score das 6 arquiteturas perante as 100 repetições

In [21]:
y = df_cat.pop(item='Cover_Type')
X = df_cat

In [ ]:
resultados = []
melhores_redes = [
    ((48, 60), 'sgd', 'relu', 150),
    ((36, 28), 'adam', 'logistic', 150),
    ((32, 32), 'sgd', 'relu', 150),
    ((17, 8), 'adam', 'logistic', 100), 
    ((12, 13), 'adam', 'relu', 100),    
    ((10, 15), 'adam', 'relu', 100)
]

**Dataset com atributos categóricos**

In [ ]:
y = df_cat.pop(item='Cover_Type')
X = df_cat

In [ ]:
# O treinamento foi executado com o seguinte comando. Em cada instância, a lista "melhores_redes" continha alguns dos elementos listados na célula acima

# resultados = (Parallel(n_jobs=-1)(delayed(avaliar_modelo)(X, y, camadas, solver, activation, iterations, n_times=100) for (camadas, solver, activation, iterations) in melhores_redes))

# with open("resultado_arquiteturas.json", "w", encoding="utf-8") as f:
#     json.dump(resultados, f, ensure_ascii=False, indent=4)

## Considerações Parciais

1. É possível identificar uma rede com desempenho superior às demais?
2. Qual estratégia mostrou-se mais producente para a obtenção de boas arquiteturas (Estratégia Própria ou Pirâmide Geométrica)? Por quê?
3. Considerar os atributos categóricos trouxe melhorias? Justifique.
4. Um número maior de épocas trouxe melhorias?
5. Qual a maior dificuldade de resolução do problema proposto perante as RNAs?